1.   **response_format** method
2.   function calling


**response_format** method was used in this notebook.

In [ ]:
! pip install -q beautifulsoup4 pydantic openai

In [ ]:
from enum import Enum
import json

import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from pydantic import BaseModel, Field

In [ ]:
from google.colab import userdata

open_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=open_api_key)
MODEL = "gpt-4o-2024-08-06"

In [ ]:
query = """
Hi, I'm having trouble with my recent order. I received the wrong item and need to return it for a refund.
Can you help me with the return process and let me know when I can expect my refund?
"""

In [ ]:
system_prompt = """
You are an AI customer care assistant. You will be provided with a customer inquiry,
and your goal is to respond with a structured solution, including the steps taken to resolve the issue and the final resolution.
For each step, provide a description and the action taken.
"""

##### JSON Schema

In [ ]:
def get_ticket_response_json(query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ticket_resolution",
                "schema": {
                    "type": "object",
                    "properties": {
                        "steps": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "description":{
                                        "type": "string",
                                        "description": "Description of the step taken.",
                                    },
                                    "action":{
                                        "type": "string",
                                        "description": "Action taken to resolve this issue.",
                                    },
                                },
                                "required": ["description", "action"],
                                "additionalProperties": False,

                            },
                        },
                        "final_resolution": {
                            "type": "string",
                            "description": "The final message taht will be send to the customer.",
                        },
                    },
                    "required": ["steps", "final_resolution"],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        },
    )

    return response.choices[0].message


response = get_ticket_response_json(query)
response.model_dump()

{'content': '{"steps":[{"description":"Verify the customer\'s order details and the item received.","action":"Check the order history to confirm the correct item ordered and identify the item that was incorrectly shipped."},{"description":"Inform the customer about the return policy and procedures.","action":"Provide the customer with the return process, including steps for sending back the wrong item."},{"description":"Generate a return shipping label for the customer.","action":"Create and send an electronic return shipping label to the customer via email."},{"description":"Guide the customer on how to pack and ship the item for return.","action":"Explain how to securely pack the item and attach the shipping label for return."},{"description":"Process a refund once the item is returned and verified.","action":"Set up a refund to be initiated as soon as the returned item is received and verified by the warehouse."}],"final_resolution":"Please follow the instructions provided in the em

In [ ]:
response_json = json.loads(response.content)
for step in response_json["steps"]:
    print(f"Step: {step['description']}")
    print(f"Action: {step['action']}\n")
print(response_json["final_resolution"])

Step: Verify the customer's order details and the item received.
Action: Check the order history to confirm the correct item ordered and identify the item that was incorrectly shipped.

Step: Inform the customer about the return policy and procedures.
Action: Provide the customer with the return process, including steps for sending back the wrong item.

Step: Generate a return shipping label for the customer.
Action: Create and send an electronic return shipping label to the customer via email.

Step: Guide the customer on how to pack and ship the item for return.
Action: Explain how to securely pack the item and attach the shipping label for return.

Step: Process a refund once the item is returned and verified.
Action: Set up a refund to be initiated as soon as the returned item is received and verified by the warehouse.

Please follow the instructions provided in the email to return the incorrect item using the prepaid shipping label. Your refund will be processed within 5-7 busines

##### Pydantic

In [ ]:
class TicketResolution(BaseModel):
    class Step(BaseModel):
        description: str = Field(description="Description of the step taken.")
        action: str = Field(description="Action taken to resolve the issue.")

    steps: list[Step]
    final_resolution: str = Field(
        description="The final message that will be send to the customer."
    )
    confidence: float = Field(description="Confidence in the resolution (0-1)")


def get_ticket_response_pydantic(query: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        response_format=TicketResolution,
    )

    return completion.choices[0].message.parsed


response_pydantic = get_ticket_response_pydantic(query)
response_pydantic.model_dump()

{'steps': [{'description': 'Verified details of the order to confirm the incorrect item was received.',
   'action': "Reviewed the customer's order history and the current order details to identify the item mismatch."},
  {'description': 'Checked the return policy concerning incorrect items received.',
   'action': "Accessed the company's return policy and procedures documentation to confirm the process for returning incorrect items."},
  {'description': 'Initiated the return process for the customer.',
   'action': 'Filled out and submitted a return request form on behalf of the customer, generating a return shipping label for them.'},
  {'description': 'Informed the customer about the return process and time expectations for a refund.',
   'action': 'Provided details to the customer on how to return the item using the shipping label and informed them about the expected time frame for the refund process.'}],
 'final_resolution': 'We apologize for the inconvenience of receiving the wro

##### Example with Enums

In [ ]:
query = "Hi there, I have a question about my bill. Can you help me?"


class TicketCategory(str, Enum):
    """Enumeration of categories for incoming tickets."""

    GENERAL = "general"
    ORDER = "order"
    RETURN = "return"
    BILLING = "billing"


# Define your desired output structure using Pydantic
class Reply(BaseModel):
    content: str = Field(description="Your reply that we send to the customer.")
    category: TicketCategory
    confidence: float = Field(
        description="Confidence in the category prediction."
    )  # ge=0, le=1,


completion = client.beta.chat.completions.parse(
    model=MODEL,
    response_format=Reply,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": query},
    ],
)

reply = completion.choices[0].message.parsed
reply.model_dump()

{'content': "Of course, I'd be happy to help with your billing question. To assist you effectively, we'll follow these steps:\n\n1. **Identify Specific Issues**\n   - Description: We'll start by identifying the specific concern you have with your bill, such as unexpected charges, incorrect amounts, or needing clarification on specific line items.\n   - Action: Please provide details about the specific issue you're experiencing with your bill.\n\n2. **Review Billing Information**\n   - Description: We'll review the billing details based on the information you provide.\n   - Action: Check the billing period, invoice details, and any associated account information that may be relevant.\n\n3. **Compare with Account Records**\n   - Description: We will compare this information with our records to verify the billing details.\n   - Action: Look at your account's transaction history, previous payments, and any applicable promotions or discounts.\n\n4. **Resolve the Issue**\n   - Description: O

##### Text summarization

In [ ]:
def get_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    html_content = soup.find("div", class_="mw-parser-output")
    content = "\n".join(p.text for p in html_content.find_all("p"))
    return content


urls = [
    # Article on CNNs
    "https://en.wikipedia.org/wiki/Convolutional_neural_network",
    # Article on LLMs
    "https://wikipedia.org/wiki/Large_language_model",
    # Article on MoE
    "https://en.wikipedia.org/wiki/Mixture_of_experts",
]

content = [get_article_content(url) for url in urls]


summarization_prompt = """
You will be provided with content from an article about an invention.
Your goal will be to summarize the article following the schema provided.
Here is a description of the parameters:
- invented_year: year in which the invention discussed in the article was invented
- summary: one sentence summary of what the invention is
- inventors: array of strings listing the inventor full names if present, otherwise just surname
- concepts: array of key concepts related to the invention, each concept containing a title and a description
- description: short description of the invention
"""


class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]


def get_article_summary(text: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        temperature=0.2,
        messages=[
            {"role": "system", "content": summarization_prompt},
            {"role": "user", "content": text},
        ],
        response_format=ArticleSummary,
    )

    return completion.choices[0].message.parsed


summaries = []

for i in range(len(content)):
    print(f"Analyzing article #{i+1}...")
    summaries.append(get_article_summary(content[i]))
    print("Done.")

Analyzing article #1...
Done.
Analyzing article #2...
Done.
Analyzing article #3...
Done.


In [ ]:
# summary of the articles
for summary in  summaries:
  print(summary)
  print("...............................")

invented_year=1980 summary='A convolutional neural network (CNN) is a type of deep learning model designed to process data with a grid-like topology, such as images, by using convolutional layers to automatically learn spatial hierarchies of features.' inventors=['Fukushima'] description='Convolutional neural networks (CNNs) are a class of deep neural networks commonly used for analyzing visual imagery. They are designed to automatically and adaptively learn spatial hierarchies of features from input data through backpropagation by using multiple building blocks, such as convolution layers, pooling layers, and fully connected layers.' concepts=[Concept(title='Convolutional Layers', description='These layers apply a convolution operation to the input, passing the result to the next layer. They are the core building blocks of CNNs, allowing the network to learn spatial hierarchies of features.'), Concept(title='Pooling Layers', description='Pooling layers reduce the dimensionality of eac